In [103]:
import pandas as pd
import numpy as np
import pickle as p

In [104]:
data_all = pd.read_csv('data_all.csv')

In [105]:
np.random.seed(42)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [106]:
item_subset

[3722, 3718, 2721, 2256, 7009, 2257, 3732, 3727, 7115, 7157]

### `/train`

In [108]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [61]:
# Model Training Result
r.json()['result']

'Success'

In [62]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,2256.0,57.811802,0.624334,13.485132,0.233259,17.465449
1,2257.0,18.398319,0.629117,5.224419,0.283962,6.918259
2,2721.0,26.516331,0.684290,10.198852,0.384625,15.537380
3,3718.0,252.716995,0.778774,38.566884,0.152609,50.760723
4,3722.0,367.495941,0.784278,56.545426,0.153867,77.688564
5,3727.0,296.811249,0.736610,49.764365,0.167663,65.884579
6,3732.0,127.388702,0.768074,20.592259,0.161649,26.882109
7,7009.0,2009.091797,0.819557,316.044212,0.157307,442.622196
8,7115.0,29.458530,0.558653,9.378256,0.318354,12.655907
9,7157.0,12.570630,0.423974,5.056628,0.402257,6.831616


### `/optimize`

In [93]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [94]:
best_prices = r.json()['result']
best_prices

{'2256': 8.81,
 '2257': 11.99,
 '2721': 12.56,
 '3718': 7.42,
 '3722': 10.19,
 '3727': 10.47,
 '3732': 10.56,
 '7009': 6.09,
 '7115': 4.92,
 '7157': 7.12}

In [95]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['pred_q']

In [96]:
qty

{'2256': 79,
 '2257': 102,
 '2721': 140,
 '3718': 486,
 '3722': 1468,
 '3727': 1500,
 '3732': 424,
 '7009': 14902,
 '7115': 100,
 '7157': 27}

In [97]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (best_prices_input[item]*qty[item])
    
# Number is obscenely high due to some outliers,
# Notably 7009, likely due to lack of constraints etc.
print(revenue)

188301.96


### `/predict/`

In [69]:
# Craft mock data input, average historical prices of items
prices_input = {}
for item in item_subset:
    avg_item_price = data_subset.query('Item_ID=={}'.format(item))['Price_'].mean()
    prices_input[str(item)] = round(avg_item_price,2)
    
prices_input

{'3722': 10.72,
 '3718': 7.6,
 '2721': 13.07,
 '2256': 8.27,
 '7009': 4.62,
 '2257': 11.54,
 '3732': 11.79,
 '3727': 10.81,
 '7115': 4.81,
 '7157': 6.66}

In [70]:
import requests
import json

url = 'http://localhost:5000/predict/'



payload = {'prices': prices_input,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [71]:
# What If Prediction
qty = r.json()['pred_q']
qty

{'2256': 57,
 '2257': 18,
 '2721': 26,
 '3718': 291,
 '3722': 361,
 '3727': 288,
 '3732': 150,
 '7009': 1922,
 '7115': 34,
 '7157': 12}

In [72]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (prices_input[item]*qty[item])
    
print(revenue)

21105.329999999998


In [101]:
with open('keys.json') as f:
    HOST_KEY = json.load(f)['host_key']